<a href="https://colab.research.google.com/github/TalissaMoura/AmazonC4DatasetAugmented/blob/main/notebooks/1_pipeline_build_hard_queries_with_one_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline for build the hard queries

# 2. Functions

In [1]:
# setup hugging faces
from huggingface_hub import login,snapshot_download, upload_file, HfApi
from google.colab import userdata
import os
HF_TOKEN = userdata.get("HF_TOKEN")
login(HF_TOKEN)
hf_api = HfApi(token=HF_TOKEN)

# 3. Load data

In [2]:
from google.colab import userdata
from huggingface_hub import snapshot_download
from datasets import load_dataset
import pandas as pd
hf_token = userdata.get('HF_TOKEN')
df_amazonc4 = load_dataset('McAuley-Lab/Amazon-C4')['test'].data.table
snapshot_download(
    repo_id='Talissa/AmazonC4Augmented',
    repo_type='dataset',
    local_dir='datasets',
    allow_patterns=['raw/review_and_meta_categories/filtered_4_star_reviews_All_Beauty.parquet'],
    token=hf_token,
)

README.md: 0.00B [00:00, ?B/s]

test.csv:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/21223 [00:00<?, ? examples/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

raw/review_and_meta_categories/filtered_(…):   0%|          | 0.00/21.4k [00:00<?, ?B/s]

'/content/datasets'

In [3]:
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq

df_filtered_reviews = pq.read_table('/content/datasets/raw/review_and_meta_categories/filtered_4_star_reviews_All_Beauty.parquet')
df_filtered_reviews

pyarrow.Table
ori_review: string
asin: string
rating: double
title: string
features: list<element: string>
  child 0, element: string
description: list<element: string>
  child 0, element: string
----
ori_review: [["Very refreshing, cleans well without over drying my skin. Is not exfoliating at all, and doesn't really lather that great, which is why I didn't give it the full five stars, but works well regardless. It rinsed off my face easily, something I appreciate. A few other cleansers seem to never fully rinse, even with a washcloth, and I hate that. This does what I need it to do without any adverse side affects, and my skin seems to be doing well with it.","Nice  brushes, I love them but my daughter thinks they are pokey. They do help remove tangled hair but still use a finning brush too.","I go to the dentist every three months due to plaque buildup and this tool has helped me to control plaque in between visits.  Very sharp so you do have to be very careful when using it.  Vibra

# 4. Build the pipeline

1 - Fetching data for apply first pipeline for list positive and negative points


In [4]:
import pyarrow as pa

df_all_reviews = df_amazonc4.select(['item_id','ori_review']).rename_columns({'ori_review':'ori_positive_review'}).join(
    df_filtered_reviews.select(['asin','ori_review']).rename_columns({'asin':'item_id','ori_review':'ori_semi_positive_review'}),
    join_type='inner',
    keys=['item_id']
)

df_final = df_all_reviews.group_by("item_id").aggregate([
    ("ori_semi_positive_review", "list"),
    ("ori_positive_review", "distinct")
])
print(df_final)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<item: string>
  child 0, item: string
ori_positive_review_distinct: list<item: string>
  child 0, item: string
----
item_id: [["B099KSC4L2","B0B34GRFL2","B0B86TKMP8","B0B186FXGM","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN","B08MF3W9DZ","B0BXXD1FZ4"]]
ori_semi_positive_review_list: [[["While a good value for your money, if you don’t have any brow hair, your pencil will not adhere to the skin, or worse look artificial."],["I like this product a lot for the vegan ingredients. it cleans without stinging my eyes. My skin feels a little more stripped than I would like it to feel. But at this price point, it's perfect. I'm just really spoiled with my skin care.<br /><br />Definitely appropriate for someone with oily skin. I will likely use it through the summer months where my workout routine might indicate that I need this. I wear make-up when I work out so I need something that really cleans afterwards. this will do the trick.","M

In [5]:
# check if we have at least 1 semi positive review and 1 one positive review for each item
from itertools import groupby
items_list = df_final.to_pylist()
for k,g in groupby(items_list, lambda x: x['item_id']):
  items = list(g)
  print(k,'qty_semi_positive_review:',[len(i['ori_semi_positive_review_list']) for i in items],'qty_positive_review:',[len(i['ori_positive_review_distinct']) for i in items])

B099KSC4L2 qty_semi_positive_review: [1] qty_positive_review: [1]
B0B34GRFL2 qty_semi_positive_review: [3] qty_positive_review: [1]
B0B86TKMP8 qty_semi_positive_review: [2] qty_positive_review: [1]
B0B186FXGM qty_semi_positive_review: [2] qty_positive_review: [1]
B08L5KN7X4 qty_semi_positive_review: [3] qty_positive_review: [1]
B00K7J4S6G qty_semi_positive_review: [1] qty_positive_review: [1]
B09TT8ZVPN qty_semi_positive_review: [3] qty_positive_review: [1]
B08MF3W9DZ qty_semi_positive_review: [2] qty_positive_review: [1]
B0BXXD1FZ4 qty_semi_positive_review: [2] qty_positive_review: [1]


2 - Fetch the llm to build the list of positive points and negative points

In [6]:
!pip install outlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00


In [7]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/system_prompt_analysis_review"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    You are a text summarizer. You are given user reviews about a product purchased online, and you need to provide a summary of the product's positive and negative attributes.

    Build two separate lists: one containing the summary about **positive aspects** and one containing the **negative aspects** regarding the product identified across **all** the reviews. Be
    as most descriptive and specific as possible. Focus on concrete features, benefits, and issues mentioned in the reviews.

    Return the results in a JSON format that has the following structure:
    {
      "positive_points": ["a point...", "another point..."],
      "negative_points": ["a point...", "another point..."]
    }
    IMPORTANT: Always complete the JSON structure. If there are no negative points, return an array with an empty string: "negative_points": [].
    Do not truncate your response - always complete the JSON object.
    """


    with open('/content/prompts/system_template_review_analysis.txt', 'w+') as f:
          f.write(system_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/system_template_review_analysis.txt",
              path_in_repo=f"prompts/system_template_review_analysis.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for review analysis"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [8]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_analysis_review"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    A super satisfied user review - 5 stars -: \n {{positive_review}} \n

    Satisfied user reviews - 4 stars -: \n {{semi_positive_reviews}} \n
    """

    with open('/content/prompts/user_prompt_analysis_review.txt', 'w') as f:
          f.write(user_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_analysis_review.txt",
              path_in_repo=f"prompts/user_prompt_analysis_review.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for review analysis"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [13]:
# Model for the response
from pydantic import BaseModel,ValidationError
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import torch
import re

class ProductReviewOutline(BaseModel):
    positive_points: list[str|None]
    negative_points: list[str|None]

path_procesed_review_analysis = pathlib.Path('/content/datasets/procesed/review_analysis')

if not pathlib.Path(path_procesed_review_analysis).exists():
  pathlib.Path.mkdir(path_procesed_review_analysis,exist_ok=True,parents=True)

MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

if "model_wrapper" not in globals():
  print("Loading model and tokenizer...")

  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,fix_mistral_regex=True,device_map="cpu",)
  model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, dtype=torch.float16,  # Use FP16 to save memory
    device_map="auto",  # Let transformers handle device mapping
     )

  model_wrapper = outlines.from_transformers(
      model,tokenizer
  )
  generator = outlines.Generator(model_wrapper,ProductReviewOutline)
  # 🔍 Validate placement
  if not next(model.parameters()).is_cuda:
      raise RuntimeError("Model failed to load on GPU!")
  print("✅ Model successfully on GPU.")
else:
    print("✅ Using existing model and tokenizer (already loaded).")

system_template = outlines.Template.from_file('/content/prompts/system_template_review_analysis.txt')
user_template = outlines.Template.from_file('/content/prompts/user_prompt_analysis_review.txt')

def extract_json_from_markdown(response_text: str) -> str:
    """Extract JSON from markdown code blocks"""
    # Look for ```json ... ``` or ``` ... ```
    pattern = r'```(?:json)?\s*(.*?)\s*```'
    matches = re.findall(pattern, response_text, re.DOTALL)

    if matches:
        # Return the first JSON block found
        return matches[0].strip()
    else:
        # If no markdown blocks, return original text
        return response_text

def clean_and_parse_json(response_text: str) -> ProductReviewOutline | None:
    # Step 1: Extract JSON from markdown if present
    json_content = extract_json_from_markdown(response_text)

    # Step 2: Remove any remaining backticks or markdown artifacts
    json_content = json_content.strip()

    # Step 3: Handle potential escaping issues
    if json_content.startswith('"') and json_content.endswith('"'):
        # Unescape if it's still a string representation
        try:
            json_content = json.loads(json_content)
        except json.JSONDecodeError:
            pass  # Keep as is if it's not double-escaped

    # Step 4: Parse and validate
    try:
        if isinstance(json_content, str):
            return ProductReviewOutline.model_validate_json(json_content)
        else:
            return ProductReviewOutline.model_validate(json_content)
    except ValidationError as e:
        print(f"Validation error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None

batches = df_final.to_batches()
for i,batch in enumerate(batches):
  # Create the parquet dataset
  new_rows = []
  for row in batch.to_pylist():
    positive_review = row['ori_positive_review_distinct']
    semi_positive_reviews = "\n".join([f"{i} - {row['ori_semi_positive_review_list'][i]}" for i in range(len(row['ori_semi_positive_review_list']))])
    messages = [
        {"role": "system", "content": system_template()},
        {"role": "user", "content": user_template(positive_review=positive_review,semi_positive_reviews=semi_positive_reviews)}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

  # Append new column for table
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=2048
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    response_treated = clean_and_parse_json(response)
    if response_treated:
      new_rows.append(response_treated)

  batch_pylist = batch.to_pylist()
  new_data = []
  for new_row,data in zip(new_rows,batch_pylist):
      print('new_row-dict:',new_row.__dict__)
      positive_points = new_row.__dict__['positive_points']
      negative_points = new_row.__dict__['negative_points']
      data['positive_points'] = positive_points
      data['negative_points'] = negative_points
      new_data.append(data)

  batch = pa.Table.from_pylist(new_data)
  pq.write_table(batch, f"/content/datasets/procesed/review_analysis/all_Beauty_review_analysis_part_{i}.parquet")
  print(f"Batch {i} done.")



✅ Using existing model and tokenizer (already loaded).
new_row-dict: {'positive_points': ['works well for eyebrows', 'love the stencils', 'mix with Maddluv', 'amazing product', 'finally solves eyebrow problems'], 'negative_points': ['does not adhere to skin if no brow hair is present', 'can look artificial if not adhered properly', 'not suitable for users without eyebrow hair']}
new_row-dict: {'positive_points': ['works well', "doesn't leave skin red", 'cleans without stinging eyes', 'cleans well and soothes the skin', 'very refreshing', 'cleans well without over drying skin', 'does not over dry skin', 'cleans well without any adverse side effects', 'my skin seems to be doing well with it'], 'negative_points': ['smells like freshly cut grass and rubbing alcohol', 'strong and medicinal', 'not my favorite', 'jelly formula', 'little chunks of Cica leaves (unnoticeable)']}
new_row-dict: {'positive_points': ['protects shampoo bar from being splashed by water', 'looks Zen in the shower', 'sm

In [ ]:
# # Delete models and tokenizer
# import gc
# import torch

# # Delete global variables
# if 'model' in globals():
#     del globals()['model']
#     print("✅ Removed 'model' from globals")

# if 'tokenizer' in globals():
#     del globals()['tokenizer']
#     print("✅ Removed 'tokenizer' from globals")

# if 'model_wrapper' in globals():
#     del globals()['model_wrapper']
#     print("✅ Removed 'model_wrapper' from globals")

# if 'generator' in globals():
#     del globals()['generator']
#     print("✅ Removed 'generator' from globals")

# # Force garbage collection to free CPU memory
# gc.collect()

# # Clear GPU cache to free up CUDA memory
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print("✅ GPU cache cleared")
# else:
#     print("CUDA not available")

# print("✅ Cleanup complete - model/tokenizer removed from CPU/GPU")

✅ GPU cache cleared
✅ Cleanup complete - model/tokenizer removed from CPU/GPU


3 - Read parquet and generate queries for the products

In [16]:
df_batch = pq.read_table('/content/datasets/procesed/review_analysis/all_Beauty_review_analysis_part_0.parquet')
print(df_batch)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<element: string>
  child 0, element: string
ori_positive_review_distinct: list<element: string>
  child 0, element: string
positive_points: list<element: string>
  child 0, element: string
negative_points: list<element: string>
  child 0, element: string
----
item_id: [["B099KSC4L2","B0B34GRFL2","B0B86TKMP8","B0B186FXGM","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN","B08MF3W9DZ","B0BXXD1FZ4"]]
ori_semi_positive_review_list: [[["While a good value for your money, if you don’t have any brow hair, your pencil will not adhere to the skin, or worse look artificial."],["I like this product a lot for the vegan ingredients. it cleans without stinging my eyes. My skin feels a little more stripped than I would like it to feel. But at this price point, it's perfect. I'm just really spoiled with my skin care.<br /><br />Definitely appropriate for someone with oily skin. I will likely use it through the summer months where my workout routin

In [34]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/system_prompt_generate_hard_negative_query"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    You are a text generator. You receive two lists: one containing positive aspects and another containing negative aspects of a product purchased online, from the users’ point of view.

    Based on these lists, generate a first-person, natural-language search query (i.e., phrased as if a real person were typing it into a search engine or asking a friend).

    The query should reflect interest in finding a product that containes one of the key positive qualities listed, but avoids or improves upon one specific negative aspect mentioned.

    IMPORTANT:
    - The query must sound conversational and realistic.
    - It must be at least 100 characters long.
    - If no negative aspects are provided (i.e., the negative list is empty), return an empty string ("").
    """


    with open('/content/prompts/system_prompt_generate_hard_negative_query.txt', 'w+') as f:
          f.write(system_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_hard_negative_query.txt",
              path_in_repo=f"prompts/system_prompt_generate_hard_negative_query.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for generate hard negative queries"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [18]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_generate_hard_negative_queries"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    positive product characteristics -: \n {{positive_list}} \n

    negative product characteristics -: \n {{negative_list}} \n
    """

    with open('/content/prompts/user_prompt_generate_hard_negative_queries.txt', 'w') as f:
          f.write(user_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_hard_negative_queries.txt",
              path_in_repo=f"prompts/user_prompt_generate_hard_negative_queries.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for generate hard negative queries"
          )


In [36]:
# Model for the response

path_procesed_hnq = pathlib.Path('/content/datasets/procesed/hard_negative_queries')

if not pathlib.Path(path_procesed_hnq).exists():
  pathlib.Path.mkdir(path_procesed_hnq,exist_ok=True,parents=True)


system_template = outlines.Template.from_file('/content/prompts/system_prompt_generate_hard_negative_query.txt')
user_template = outlines.Template.from_file('/content/prompts/user_prompt_generate_hard_negative_queries.txt')

batches = df_batch.to_batches()
for i,batch in enumerate(batches):
  # Create the parquet dataset
  new_rows = []
  for row in batch.to_pylist():
    positive_list = "-".join(row['positive_points'])
    negative_list = "-".join(row['negative_points'])
    queries = []
    for _ in range(3):
      messages = [
          {"role": "system", "content": system_template()},
          {"role": "user", "content": user_template(positive_list=positive_list,
                                                    negative_list=negative_list)}
      ]
      text = tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
      )

      ## Append new column for table
      model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

      generated_ids = model.generate(
          **model_inputs,
          max_new_tokens=2048
      )
      generated_ids = [
          output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
      ]

      response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
      print(response)
      if len(response)>=100:
        queries.append(response)

    new_rows.append(queries)

  batch_pylist = batch.to_pylist()
  new_data = []
  for new_row,data in zip(new_rows,batch_pylist):
      data["hard_negative_query"] = new_row
      new_data.append(data)

  batch = pa.Table.from_pylist(new_data)
  pq.write_table(batch, f"/content/datasets/procesed/hard_negative_queries/all_Beauty_part_{i}.parquet")
  print(f"Batch {i} done.")



looking for an eyebrow product that uses Maddluv stencils but doesn't look too artificial when there's no natural brow hair
I'm looking for an eyebrow product that's amazing and uses Maddluv, but I want one that doesn't always look artificial if there's no natural brow hair.
I'm looking for an eyebrow pomade that doesn't have that artificial look when there's no brow hair, and I love the stencils too! Something that can mix well with other products like Maddluv.
I'm looking for a gentle cleanser that doesn't leave my skin red, but I want to avoid the strong smell and medicinal taste. Does anyone have recommendations for a good one?
I'm looking for a skincare product that cleanses well and soothes the skin without leaving it red or feeling over-dried. I hate the strong smell and medicinal taste though; does anyone have any recommendations for something similar but with a milder scent?
looking for a skincare product that cleanses well without leaving my skin red but doesn't have that str

In [37]:
df_batch_hq = pq.read_table('/content/datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet')
print(df_batch_hq)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<element: string>
  child 0, element: string
ori_positive_review_distinct: list<element: string>
  child 0, element: string
positive_points: list<element: string>
  child 0, element: string
negative_points: list<element: string>
  child 0, element: string
hard_negative_query: list<element: string>
  child 0, element: string
----
item_id: [["B099KSC4L2","B0B34GRFL2","B0B86TKMP8","B0B186FXGM","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN","B08MF3W9DZ","B0BXXD1FZ4"]]
ori_semi_positive_review_list: [[["While a good value for your money, if you don’t have any brow hair, your pencil will not adhere to the skin, or worse look artificial."],["I like this product a lot for the vegan ingredients. it cleans without stinging my eyes. My skin feels a little more stripped than I would like it to feel. But at this price point, it's perfect. I'm just really spoiled with my skin care.<br /><br />Definitely appropriate for someone with oily skin. 